## MCP Weather & Air Quality Explorer (No-Key, Open Tools)

### What is MCP?

MCP (Model Context Protocol) is an open protocol that defines a standardized way for AI clients to interact with external tools, data sources, and services.

Instead of hardcoding APIs, MCP allows a client to:

* Discover available tools dynamically

* Call tools using a common protocol

* Receive structured, machine-readable responses

* Remain independent of specific implementations

In MCP, a server exposes tools (such as weather lookup, air quality queries, or data extraction), and a client communicates with that server using a standardized JSON-RPC-based interface.

This makes MCP ideal for building modular, reusable, and tool-agnostic AI workflows.

**Purpose of This Notebook**

This notebook demonstrates how to use MCP in practice by connecting to a publicly available, no-API-key MCP weather server.

**You will learn how to:**

* Start an MCP server locally

* Perform the MCP handshake lifecycle (initialize, initialized)

* Discover tools dynamically using tools/list

* Invoke tools using tools/call

* Parse natural-language tool responses

* Convert unstructured outputs into structured tables

* Query multiple cities in batch

* Retrieve both weather and air quality data


The results are parsed and normalized into a clean dataframe.

**Find No-Key MCP Weather Servers from the Awesome MCP List**

This script:

Downloads the awesome-mcp-servers README

Finds repositories related to weather

Fetches each repo’s README

Filters for servers that explicitly mention “no API key required”

In [1]:
import re, requests

AWESOME_RAW = "https://raw.githubusercontent.com/punkpeye/awesome-mcp-servers/main/README.md"

WEATHER_HINTS = [
    "weather", "forecast", "wttr", "open-meteo", "meteomatics", "meteo", "temperature"
]

NO_KEY_PATTERNS = [
    r"no api key required",
    r"no\s+api\s+keys?\s+required",
    r"does not require (an )?api key",
    r"without (an )?api key",
    r"api key\s+not\s+required",
]

def get_text(url):
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    return r.text

def all_repo_slugs_from_markdown(md: str):
    # Extract owner/repo from any github.com/owner/repo links
    return sorted(set(re.findall(r"https?://github\.com/([A-Za-z0-9_.-]+/[A-Za-z0-9_.-]+)", md)))

def find_weather_candidates(md: str):
    candidates = set()
    for line in md.splitlines():
        low = line.lower()
        if any(h in low for h in WEATHER_HINTS) and "github.com/" in low:
            for slug in re.findall(r"https?://github\.com/([A-Za-z0-9_.-]+/[A-Za-z0-9_.-]+)", line):
                candidates.add(slug)
    return sorted(candidates)

def repo_readme_raw(owner_repo):
    for branch in ("main", "master"):
        url = f"https://raw.githubusercontent.com/{owner_repo}/{branch}/README.md"
        try:
            return get_text(url), url
        except Exception:
            pass
    return None, None

def is_no_key(readme_text: str):
    t = readme_text.lower()
    return any(re.search(p, t) for p in NO_KEY_PATTERNS)

# 1) Load awesome list
awesome_md = get_text(AWESOME_RAW)

# 2) Find weather-ish candidates from lines
candidates = find_weather_candidates(awesome_md)

# 3) Verify "no key" by scanning each repo README
no_key_weather = []
for slug in candidates:
    readme, readme_url = repo_readme_raw(slug)
    if readme and is_no_key(readme):
        no_key_weather.append((slug, readme_url))

print("No-key MCP weather servers found:")
for slug, readme_url in no_key_weather:
    print("-", slug, "|", readme_url)


No-key MCP weather servers found:
- isdaniel/mcp_weather_server | https://raw.githubusercontent.com/isdaniel/mcp_weather_server/main/README.md


**Install Pandas and MCP dependencies**

In [2]:
!pip -q install mcp pandas requests


In [3]:
! pip -q install mcp-weather-server


**Install Dependencies & Clone MCP Weather Server**

In [4]:
!pip -q install mcp pandas requests
!git clone -q https://github.com/isdaniel/mcp_weather_server.git
%cd mcp_weather_server
!pip -q install -r requirements.txt || true



fatal: destination path 'mcp_weather_server' already exists and is not an empty directory.
/content/mcp_weather_server


##Start MCP Server & Initialize Client (stdio)

### This cell:

*Launches the MCP weather server

*Implements a robust JSON-RPC stdio client

*Performs the required MCP handshake (initialize + initialized)

In [5]:
import json
import subprocess
import time
from typing import Any, Dict, Optional

server = subprocess.Popen(
    ["python", "-m", "mcp_weather_server"],  # if this fails, see Cell 6
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True,
    bufsize=1,
)

def _read_jsonrpc_line(timeout_s: float = 10.0) -> Dict[str, Any]:
    start = time.time()
    while True:
        if time.time() - start > timeout_s:
            raise TimeoutError("Timed out waiting for JSON-RPC response.")
        line = server.stdout.readline()
        if not line:
            raise RuntimeError("Server stdout closed (server may have crashed).")
        line = line.strip()
        if not line:
            continue
        try:
            obj = json.loads(line)
        except json.JSONDecodeError:
            # Some servers accidentally log to stdout; ignore non-JSON lines.
            continue
        if isinstance(obj, dict) and obj.get("jsonrpc") == "2.0":
            return obj

def rpc(method: str, params: Optional[Dict[str, Any]] = None, _id: int = 1) -> Any:
    req = {"jsonrpc": "2.0", "id": _id, "method": method}
    if params is not None:
        req["params"] = params
    server.stdin.write(json.dumps(req) + "\n")
    server.stdin.flush()
    resp = _read_jsonrpc_line()
    if "error" in resp:
        raise RuntimeError(resp["error"])
    return resp["result"]

def notify(method: str, params: Optional[Dict[str, Any]] = None) -> None:
    req = {"jsonrpc": "2.0", "method": method}
    if params is not None:
        req["params"] = params
    server.stdin.write(json.dumps(req) + "\n")
    server.stdin.flush()

# REQUIRED: protocolVersion inside params
init_result = rpc(
    "initialize",
    {
        "protocolVersion": "2025-03-26",  # server may respond with a different supported version
        "capabilities": {},
        "clientInfo": {"name": "colab-client", "version": "0.1"},
    },
    _id=1,
)

# REQUIRED: initialized notification after initialize
notify("initialized")

init_result


{'protocolVersion': '2025-03-26',
 'capabilities': {'experimental': {}, 'tools': {'listChanged': False}},
 'serverInfo': {'name': 'mcp-weather-server', 'version': '1.24.0'}}

##Discover Available MCP Tools

In this step, we query the MCP server to retrieve the list of all tools it exposes. This allows us to dynamically inspect what capabilities the server provides without hardcoding any tool names.

What This Does

Requests the tool registry from the MCP server

Extracts the name of each available tool

Displays them as a simple list

### Why This Is Important

MCP servers can expose different tools depending on their implementation. Instead of assuming tool names, we:

* Stay compatible with different servers
* Enable dynamic discovery
* Avoid hardcoding
* Improve reliability

After this step, you will have a list of tool names such as:

* get_current_weather

* get_weather_details

* get_air_quality

In [6]:
tools = rpc("tools/list", {}, _id=2)

tool_names = [t["name"] for t in tools.get("tools", [])]
tool_names


['get_current_weather',
 'get_weather_byDateTimeRange',
 'get_weather_details',
 'get_current_datetime',
 'get_timezone_info',
 'convert_time',
 'get_air_quality',
 'get_air_quality_details']

In [7]:
cities = ["Bengaluru", "Mumbai", "Delhi", "Chennai", "Kolkata"]


### Retrieve and Normalize Weather Data for Multiple Cities

In this step, we use the selected MCP weather tool to fetch current weather information for a list of cities. The raw responses from the MCP server are then parsed and transformed into a structured tabular format.

#### What This Step Does

This stage of the notebook performs the following operations:

1. Selects the Weather Tool

The tool get_current_weather is chosen from the list of tools previously discovered from the MCP server. This ensures that the client only calls valid, server-supported capabilities.

2. Sends MCP Tool Calls

For each city in the input list:

A request is sent to the MCP server using the tools/call method.

The city name is passed as an argument.

The server returns a natural-language weather description.

These are true MCP protocol calls, not local function invocations.

3. Extracts the Text Response

MCP tool responses often contain multiple content blocks. This step extracts only the human-readable text from the response so it can be parsed reliably.

4. Parses Temperature and Weather Condition

From the raw text, two key values are extracted:

Temperature (for example: 23°C, 28.5°C)

Weather condition (for example: “Mostly clear”, “Light rain”)

Pattern matching is used to normalize these values across different responses.

5. Builds a Structured Table

The extracted data is stored in a table with the following columns:

Column	Description
City	Name of the city
Temperature	Current temperature
Weather	Current weather condition

This format makes the data easy to display, export, or analyze.

In [8]:
import re
import pandas as pd

tool_name = "get_current_weather"  # from your tool list

def call_tool(tool: str, arguments: dict, _id: int):
    # uses rpc() from earlier cells
    return rpc("tools/call", params={"name": tool, "arguments": arguments}, _id=_id)

def extract_text(tool_call_result):
    # MCP tool results often return: {"content":[{"type":"text","text":"..."}]}
    if isinstance(tool_call_result, dict) and "content" in tool_call_result:
        parts = []
        for item in tool_call_result["content"]:
            if item.get("type") == "text" and "text" in item:
                parts.append(item["text"])
        return "\n".join(parts).strip()
    return str(tool_call_result)

def parse_temp_and_weather(raw: str):
    # Temperature: look for "22.5°C" or "22 C"
    temp = None
    m = re.search(r"(-?\d+(?:\.\d+)?)\s*°?\s*C", raw, re.IGNORECASE)
    if m:
        temp = f"{m.group(1)}°C"

    # Weather/condition: try to capture a phrase like "Mainly clear", "Light rain", etc.
    # Common phrasing: "<condition> with a temperature of X°C"
    condition = None
    m2 = re.search(r"^(.+?)\s+with\s+(?:an?|a)\s+temperature", raw.strip(), re.IGNORECASE)
    if m2:
        condition = m2.group(1).strip()
    else:
        # fallback: first sentence up to period
        m3 = re.search(r"^(.+?)(?:\.|$)", raw.strip())
        condition = m3.group(1).strip() if m3 else None

    return temp, condition

rows = []
for i, city in enumerate(cities, start=1):
    try:
        out = call_tool(tool_name, {"city": city}, _id=100 + i)
        raw = extract_text(out)
        temp, condition = parse_temp_and_weather(raw)
        rows.append({"City": city, "Temperature": temp, "Weather": condition})
    except Exception as e:
        rows.append({"City": city, "Temperature": None, "Weather": None, "Error": str(e)})

df = pd.DataFrame(rows)
df


,City,Temperature,Weather
0,Bengaluru,23.4°C,The weather in Bengaluru is Overcast
1,Mumbai,26.4°C,The weather in Mumbai is Clear sky
2,Delhi,13.2°C,The weather in Delhi is Clear sky
3,Chennai,27.2°C,The weather in Chennai is Overcast
4,Kolkata,21.6°C,The weather in Kolkata is Clear sky


In [9]:
df[["City", "Temperature", "Weather"]]


,City,Temperature,Weather
0,Bengaluru,23.4°C,The weather in Bengaluru is Overcast
1,Mumbai,26.4°C,The weather in Mumbai is Clear sky
2,Delhi,13.2°C,The weather in Delhi is Clear sky
3,Chennai,27.2°C,The weather in Chennai is Overcast
4,Kolkata,21.6°C,The weather in Kolkata is Clear sky


In [10]:
import pprint

aq_tools = ["get_air_quality", "get_air_quality_details"]

# Print schema so you can confirm argument names and output format
for t in tools.get("tools", []):
    if t["name"] in aq_tools:
        print(f"\n=== {t['name']} schema ===")
        pprint.pprint(t)



=== get_air_quality schema ===
{'description': 'Get air quality information for a specified city including '
                'PM2.5, PM10,\n'
                '            ozone, nitrogen dioxide, carbon monoxide, and '
                'other pollutants. Provides health\n'
                '            advisories based on current air quality levels.',
 'inputSchema': {'properties': {'city': {'description': 'The name of the city '
                                                        'to fetch air quality '
                                                        'information for, '
                                                        'PLEASE NOTE English '
                                                        'name only, if the '
                                                        "parameter city isn't "
                                                        'English please '
                                                        'translate to English '
                     

### Tool call for Air Quality

In [11]:
aq_variables = [
    "pm2_5",
    "pm10",
    "nitrogen_dioxide",
    "ozone",
    "sulphur_dioxide",
    "carbon_monoxide",
]


In [12]:
import re
import pandas as pd

def call_tool(tool: str, arguments: dict, _id: int):
    return rpc("tools/call", params={"name": tool, "arguments": arguments}, _id=_id)

def extract_text(tool_call_result):
    if isinstance(tool_call_result, dict) and "content" in tool_call_result:
        parts = []
        for item in tool_call_result["content"]:
            if item.get("type") == "text" and "text" in item:
                parts.append(item["text"])
        return "\n".join(parts).strip()
    return str(tool_call_result)

def parse_aqi_best_effort(raw: str):
    aqi = None
    cat = None

    m = re.search(r"\bAQI\b\s*[:=]?\s*(\d{1,4})\b", raw, re.IGNORECASE)
    if m:
        aqi = int(m.group(1))

    mcat = re.search(
        r"\b(Good|Moderate|Unhealthy(?:\s+for\s+Sensitive\s+Groups)?|Unhealthy|Very\s+Unhealthy|Hazardous)\b",
        raw,
        re.IGNORECASE,
    )
    if mcat:
        cat = mcat.group(1)

    return aqi, cat

rows = []
for i, city in enumerate(cities, start=1):
    row = {"City": city}

    # 1) Summary air quality (often returns AQI-ish summary text)
    try:
        aq = call_tool("get_air_quality", {"city": city}, _id=200 + i)
        aq_text = extract_text(aq)
        aqi, aqi_cat = parse_aqi_best_effort(aq_text)
        row["AQI"] = aqi
        row["AQI_Category"] = aqi_cat
        row["Air_Quality_Summary_Raw"] = aq_text
    except Exception as e:
        row["AQI"] = None
        row["AQI_Category"] = None
        row["Air_Quality_Summary_Raw"] = None
        row["Air_Quality_Summary_Error"] = str(e)

    # 2) Detailed pollutants (variables)
    try:
        aqd = call_tool(
            "get_air_quality_details",
            {"city": city, "variables": aq_variables},
            _id=300 + i
        )
        aqd_text = extract_text(aqd)

        # Best-effort parsing for each requested variable
        # Works if the server prints like "pm2_5: 12.3" etc.
        for var in aq_variables:
            m = re.search(rf"\b{re.escape(var)}\b\s*[:=]?\s*([0-9]+(?:\.[0-9]+)?)", aqd_text, re.IGNORECASE)
            row[var] = float(m.group(1)) if m else None

        row["Air_Quality_Details_Raw"] = aqd_text
    except Exception as e:
        for var in aq_variables:
            row[var] = None
        row["Air_Quality_Details_Raw"] = None
        row["Air_Quality_Details_Error"] = str(e)

    rows.append(row)

df_aq = pd.DataFrame(rows)

# Compact view
cols = ["City", "AQI", "AQI_Category"] + aq_variables
df_aq[cols]


,City,AQI,AQI_Category,pm2_5,pm10,nitrogen_dioxide,ozone,sulphur_dioxide,carbon_monoxide
0,Bengaluru,None,Good,None,None,None,None,None,None
1,Mumbai,None,Good,None,None,None,None,None,None
2,Delhi,None,Good,None,None,None,None,None,None
3,Chennai,None,Good,None,None,None,None,None,None
4,Kolkata,None,Good,None,None,None,None,None,None


In [13]:
df_aq[["City", "Air_Quality_Summary_Raw", "Air_Quality_Details_Raw"]]


,City,Air_Quality_Summary_Raw,Air_Quality_Details_Raw
0,Bengaluru,Please analyze the following JSON air quality ...,"{\n ""city"": ""Bengaluru"",\n ""latitude"": 12.97..."
1,Mumbai,Please analyze the following JSON air quality ...,"{\n ""city"": ""Mumbai"",\n ""latitude"": 19.07283..."
2,Delhi,Please analyze the following JSON air quality ...,"{\n ""city"": ""Delhi"",\n ""latitude"": 28.65195,..."
3,Chennai,Please analyze the following JSON air quality ...,"{\n ""city"": ""Chennai"",\n ""latitude"": 13.0878..."
4,Kolkata,Please analyze the following JSON air quality ...,"{\n ""city"": ""Kolkata"",\n ""latitude"": 22.5626..."
